<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Данные
`test.tsv` - тестовая выборка. В каждой строке через табуляцию записаны: *<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...*

Будем использовать предобученные векторные представления слов [GoogleNews-vectors-negative300](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit) (<b>скачаем их</b>), которые были обучены с помощью стандартной модели word2vec на данных Google News (100 миллиардов слов). Модель содержит 300-мерные вектора для 3 миллионов слов и фраз

<b>Загрузим</b> их после <b>скачивания</b> с помощью функции [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) библиотеки Gensim, с которой вы познакомились на семинаре. Загрузим только часть векторов, указав параметр *limit* = 500000.

In [ ]:
#!gdown https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&export=download

In [ ]:
!wget -c 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'

--2020-04-09 13:03:48--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.228.163
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.228.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  2.63MB/s    in 16m 52s 

2020-04-09 13:20:41 (1.55 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
!pip install -U gensim

     |████████████████████████████████| 24.2 MB 300 kB/s eta 0:00:011
     |████████████████████████████████| 105 kB 13.1 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 9.9 MB/s eta 0:00:01
     |████████████████████████████████| 6.1 MB 8.3 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 2.0 MB/s eta 0:00:011
     |████████████████████████████████| 547 kB 5.5 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-1.11.1-py3-none-any.whl size=95255 sha256=616b628a97b889976fb50514992293f8f37b86fc05be733ca0164d672b4e76e4
  Stored in directory: /home/biomega/.cache/pip/wheels/1a/8c/a2/7b24df77c58dab0aec275c1bd3d4392d54df941020199f6759
Successfully built smart-open
  Attempting uninstall: docutils
    Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16


In [ ]:
import gensim

unable to import 'smart_open.gcs', disabling that module


In [ ]:
wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format(fname='GoogleNews-vectors-negative300.bin.gz', limit=500000, binary=True)

### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (300,)


<b>Еще раз напомню, что семинар нужно прочитать

Найдем наиболее близкие слова к слову `dog`:

In [ ]:
wv_embeddings.most_similar(word)[:5]

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('dogs', 0.8680489659309387),
 ('puppy', 0.8106428384780884),
 ('pit_bull', 0.780396044254303),
 ('pooch', 0.7627377510070801),
 ('cat', 0.7609456777572632)]

### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`?

In [ ]:
(wv_embeddings.similar_by_word('cat'))[:5]

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cats', 0.8099379539489746),
 ('dog', 0.7609456777572632),
 ('kitten', 0.7464985251426697),
 ('feline', 0.7326233983039856),
 ('beagle', 0.7150583267211914)]

# ***Входит***

---





### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()
import re

In [ ]:
def question_to_vec(question, embeddings, dim=300):

    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    # убрать знак вопроса, если он есть
    words = re.sub('[,\.?!]',' ',question).split()
    n_known = 0
    result = np.array([0] * dim, dtype=float)
    
    for word in words:
        if word in embeddings:
            result += embeddings[word]
            n_known += 1
            
    if n_known != 0:
        return result/n_known
    else:
        return result

Теперь у нас есть метод для создания векторного представления любого предложения.

In [ ]:
question_to_vec('does it works?', wv_embeddings)

array([ 0.11905924,  0.05058225,  0.08186849,  0.08262126, -0.13997396,
        0.09000651,  0.10310872, -0.1023763 , -0.01074219, -0.0933431 ,
       -0.1656901 , -0.09619141, -0.10457357, -0.08393351, -0.13720703,
        0.14233398,  0.09749349,  0.12613932,  0.16178385, -0.11080933,
       -0.1262207 ,  0.12972005, -0.03141276, -0.00325521,  0.04451497,
       -0.07104492, -0.17333984,  0.10603841, -0.19533285, -0.0234375 ,
       -0.0616862 ,  0.09033203,  0.0069987 , -0.03613281,  0.09602865,
       -0.03564707, -0.02850342,  0.10120646,  0.01953125, -0.04231771,
        0.08984375,  0.06083171,  0.11914062, -0.00447591, -0.07080078,
       -0.00358073, -0.11368815, -0.02696737,  0.07613118, -0.12084961,
       -0.00708008,  0.00641886,  0.08581543, -0.05395508,  0.07006836,
        0.0275472 , -0.0639801 , -0.06656901,  0.05677287, -0.02193705,
        0.1354777 ,  0.12190755, -0.15869141, -0.11604818, -0.05000814,
       -0.02450562, -0.02506765,  0.01011149, -0.11344401, -0.03

In [ ]:
question_to_vec('asdaf ?', wv_embeddings)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за низкую позицию корректного ответа

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. *"Как узнать с++"*
2. <font color='red'>*"Что такое язык python"*</font>
3. *"Хочу учить Java"*
4. *"Не понимаю Tensorflow"*

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов $rank\_q_i^{'}$(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
    """
        result: вернуть  Hits@k
    """
    N = len(dup_ranks)
    hits_value = 0
    for i in dup_ranks:
      if i <= k:
        hits_value += 1
    return hits_value/N    

In [ ]:
def dcg_score(dup_ranks, k):
    """
        result: вернуть DCG@k
    """
    new_dup_ranks = np.array(dup_ranks)
    N = len(dup_ranks)
    dcg_value = 0
    for i in new_dup_ranks:
      if i <= k:
        dcg_value += 1/np.log2((1+i))
    return dcg_value/N

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown"]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"]]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [candidates_ranking[i].index(copy_answers[i]) + 1 for i in range(len(copy_answers))]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks,k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Ранжирование вопросов StackOverflow

- *тестовая* выборка (test.tsv) содержит в каждой строке: *вопрос, похожий вопрос, отрицательный пример 1, отрицательный пример 2, ...*
TEST!!!

Считаем тестовую выборку для оценки качества текущего решения.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

In [ ]:
!gdown https://drive.google.com/uc?id=12VAWjs-kvpgapurkuURus7E53F711uFV&export=download
# или скачайте по ссылке https://drive.google.com/uc?id=12VAWjs-kvpgapurkuURus7E53F711uFV

Downloading...
From: https://drive.google.com/uc?id=12VAWjs-kvpgapurkuURus7E53F711uFV
To: /content/test.tsv
201MB [00:01, 105MB/s] 


In [ ]:
test = read_corpus('test.tsv')

Кол-во строк

In [ ]:
len(test)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(test[0]))

1 1001
2 1001
3 1001
4 1001
5 1001


Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список *[(2, c), (0, a), (1, b)]*.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, dim=300):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    vec_question = question_to_vec(question, embeddings, dim)
    vec_candidates = np.array([question_to_vec(candidates[i], embeddings, dim)
                      for i in range(len(candidates))])
    rank_candidates = np.array([(i, candidates[i]) for i in range(len(candidates))])
    # ранжирование
    dist_s = cosine_similarity(vec_candidates, np.array([vec_question]))[:, 0]
    return deepcopy(rank_candidates[dist_s.argsort()[::-1]])

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, 300)
        print(ranks)
        print()


[['1' 'C# create cookie from string and send it']
 ['0' 'Convert Google results object (pure js) to Python object']
 ['2' 'How to use jQuery AJAX for an outside domain?']]

[['0' 'Getting all list items of an unordered list in PHP']
 ['2' 'select2 not displaying search results']
 ['1' 'WPF- How to update the changes in list item of a list']]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(0, 'Getting all list items of an unordered list in PHP'),
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для экспереминта 1`  1, 0, 2. Для второго экперимента вы знаете один индекс уже.

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут).

In [ ]:
wv_ranking = []
for line in test:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index('0') + 1)

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.235 | Hits@   1: 0.235
DCG@   5: 0.289 | Hits@   5: 0.337
DCG@  10: 0.307 | Hits@  10: 0.392
DCG@ 100: 0.348 | Hits@ 100: 0.596
DCG@ 500: 0.377 | Hits@ 500: 0.827
DCG@1000: 0.395 | Hits@1000: 1.000


Если вы проделали все шаги правильно, то вы должны немного разочароваться полученными результатами. Давайте попробуем понять, почему качество модели такое низкое. Когда вы работаете с какими-либо данными, очень полезно первым делом посмотреть на них глазами. Выведим несколько вопросов из наших данных:

In [ ]:
for line in test[:3]:
    q, *examples = line
    print(q, *examples[:3])
    print()

How to print a binary heap tree without recursion? How do you best convert a recursive function to an iterative one? How can i use ng-model with directive in angular js flash: drawing and erasing

How to start PhoneStateListener programmatically? PhoneStateListener and service Java cast object[] to model WCF and What does this mean?

jQuery: Show a div2 when mousenter over div1 is over when hover on div1 depenting on if it is on div2 or not it should act differently How to run selenium in google app engine/cloud? Python Comparing two lists of strings for similarities



Как вы можете заметить, мы имеем дело с сырыми данными. Это означает, что там присутствует много опечаток, спецсимволов и заглавных букв. В нашем случае это все может привести к ситуации, когда для данных токенов нет предобученных векторов. Поэтому необходима предобработка.

Реализуйем функцию предобработки текстов. Вам требуется:
- Перевести символы в нижний регистр;
- Заменить символы пунктуации на пробелы;
- Удалить "плохие" символы;
- Удалить стопслова.

In [ ]:
import re
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/biomega/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Кол-во стоп стопслов

In [ ]:
stopWords = set(stopwords.words('english'))
len(stopWords)

179

In [ ]:
def text_prepare(text):
    """
        text: a string
        
        return: modified string
    """
    # Перевести символы в нижний регистр
    text = text.lower()
    
    # Заменить символы пунктуации на пробелы
    text = re.sub(r'[{}]'.format(string.punctuation), ' ', text)
    
    # Удалить "плохие" символы
    text = re.sub('[^A-Za-z0-9 ]', '', text)
    
    # Удалить стопслова.
    stopWords = set(stopwords.words('english'))
    for stopWord in stopWords:
        text = re.sub(r'\b{}\b'.format(stopWord), '', text)
    return text

Теперь преобразуйте все вопросы из тестовой выборки. Оцените, как изменилось качество. Сделайте выводы. Для изменения текста понадобиться около 30 минут.

In [ ]:
from tqdm import tqdm_notebook
from copy import deepcopy
new_test = deepcopy(test)
for i in range(len(test)):
    for j in range(len(test[i])):
        new_test[i][j] = text_prepare(new_test[i][j])

In [ ]:
wv_ranking = []
for line in new_test:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index('0') + 1)

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.340 | Hits@   1: 0.340
DCG@   5: 0.412 | Hits@   5: 0.477
DCG@  10: 0.427 | Hits@  10: 0.523
DCG@ 100: 0.463 | Hits@ 100: 0.702
DCG@ 500: 0.482 | Hits@ 500: 0.855
DCG@1000: 0.498 | Hits@1000: 1.000


# Finally!.. Visualization! (Again..)

Раз уж мы научились получать эмбеддинги предложений, а не только слов, давайте попробуем визуализировать эмбеддинги предложений!

Функция получения эмбеддинга по предложению у нас уже есть (question_to_vec в начале ноутбука). Нам осталось выбрать, какой датасет мы будем использовать (quora.txt с семинара или stackoverflow из этого дз), и далее:

1. Проделать предобработку вопросов (text_prepare, как выше)
2. Для всех вопросов получить эмбединнги (question_to_vec)
3. Применить к массиву эмбеддингов TSNE (как на семинаре)
4. Не забыть нормализовать векторы, полученные из TSNE
5. Запустить функцию draw из семинара!

Извлечем векторы, векторы вопросов, вопросы и сохраним это

In [ ]:
#make list of vectros
vectors=[]
for line in new_test:
  q, *ex = line
  vectors.append(question_to_vec(q, wv_embeddings))

In [ ]:
vect=pd.DataFrame(vectors)
vect.to_csv('vect.csv')

In [ ]:
#list of qustions
questions_l=[]
for line in new_test:
  q, *ex = line
  questions_l.append(q)

In [ ]:
quest=pd.DataFrame(questions_l)
quest.to_csv('quest.csv')

In [ ]:
import pandas as pd
df=pd.DataFrame(new_test)

In [ ]:
df.to_csv('full.csv')

Загрузка данных

In [ ]:
import pandas as pd
questions_p = pd.read_csv('quest.csv')

In [ ]:
questions_l = questions_p['0'].values.tolist()

In [ ]:
vectors=[]
for line in questions_l:
    vectors.append(question_to_vec(line, wv_embeddings))

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2, verbose=50)
#tsne.fit(vectors)

In [ ]:
word_vectors_tsne = tsne.fit_transform(vectors)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 3760 samples in 0.084s...
[t-SNE] Computed neighbors for 3760 samples in 6.474s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3760
[t-SNE] Computed conditional probabilities for sample 2000 / 3760
[t-SNE] Computed conditional probabilities for sample 3000 / 3760
[t-SNE] Computed conditional probabilities for sample 3760 / 3760
[t-SNE] Mean sigma: 0.370917
[t-SNE] Computed conditional probabilities in 0.231s
[t-SNE] Iteration 50: error = 88.0160675, gradient norm = 0.1457386 (50 iterations in 32.562s)
[t-SNE] Iteration 100: error = 88.5552902, gradient norm = 0.1365113 (50 iterations in 37.562s)
[t-SNE] Iteration 150: error = 88.0327377, gradient norm = 0.1776021 (50 iterations in 39.617s)
[t-SNE] Iteration 200: error = 89.0436172, gradient norm = 0.1131911 (50 iterations in 37.728s)
[t-SNE] Iteration 250: error = 89.7128906, gradient norm = 0.1327003 (50 iterations in 36.207s)
[t-SNE] KL divergence after 250 it

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler().fit(word_vectors_tsne)
word_vectors_tsne = ss.transform(word_vectors_tsne)

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [ ]:
output_notebook()
draw_vectors(word_vectors_tsne[:, 0], word_vectors_tsne[:, 1], token=questions_l)

Loading BokehJS ...

Figure(id='1006', ...)

Отлично. Но, к сожалению, визуализацию тестами на канвасе проверить нельзя, поэтому давайте еще напишем функцию, которая будет находить к вопросу ближайшие похожие =)

In [ ]:
def find_closest_questions(question, k=5):
    """
    function that finds closest questions from dataset given question
    args:
        question: question, preprocessed using text_prepare 
        k: how many nearest questions to find
    """

    vec_question = question_to_vec(question, wv_embeddings)[None,...]
    dist_s = cosine_similarity(vectors, vec_question)
    dist_s = np.squeeze(dist_s, 1)
    dist_args = dist_s.argsort()[::-1]

    frame = pd.DataFrame(index=range(k), columns=['questions','distance'])
    for i in range(k):
      frame.at[i, 'questions'] = questions_l[dist_args[i]]
      #print(i, dist_args[k])
      frame.at[i, 'distance'] = dist_s[dist_args[i]]
    
    
    return frame

In [ ]:
find_closest_questions(text_prepare("Why am I so stupid?"))

,questions,distance
0,hello guys getting error submitting simp...,0.521893
1,wpdb insert big problem weird,0.511799
2,uiscrollview zoomtorect animated weird beh...,0.437201
3,xmlhttprequest response type wrong firefox,0.427939
4,javascript value composed several words valu...,0.423086


### Вопрос 10:
* Какой самый ближайший вопрос к "Why am I so stupid?"?
В канвас напишите слова вопроса с маленькой буквы через пробелы без знаков пунктуации (только латинские буквы)

hello guys    getting  error  submitting  simple form   honestly   know  